In [1]:
import os
import warnings
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
warnings.filterwarnings('ignore')

os.environ["TAVILY_API_KEY"]=TAVILY_API_KEY
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition

tool = TavilySearchResults(max_results=2)
tools=[tool]
tool_node = ToolNode(tools)

In [5]:
import json
from langchain_core.messages import ToolMessage
class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage"""
    def __init__(self, tools:list) -> None:
        self.tools_by_name={tool.name: tool for tool in tools}
    
    def __call__(self, inputs:dict):
        if messages := inputs.get("messages",[]):
            message = messages[-1]
        else:
            raise ValueError("no message found in input")
        outputs=[]
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"]
                )
            )
        return {"messages": outputs}

In [4]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o-mini")
llm_with_toold = llm.bind_tools(tools)

def chatbot(state: State):
    result=llm_with_toold.invoke(state["messages"])
    return {"messages": [result]}

In [5]:
from langgraph.graph import StateGraph
graph_builder=StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph=graph_builder.compile()

In [6]:
graph.invoke({"messages": {"role": "user", "content": "지금 한국 대통령은 누구야?"}})

{'messages': [HumanMessage(content='지금 한국 대통령은 누구야?', additional_kwargs={}, response_metadata={}, id='104d25e1-0ef9-465f-8b2d-5b010f235f52'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nLqzRC4SHgcFLOYygzzfGOnF', 'function': {'arguments': '{"query":"현재 한국 대통령"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 87, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1e9111b9-79b5-41dc-abab-6c97ca3a2008-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '현재 한국 대통령'}, 'id': 'call_nLqzRC4SHgcFLOYygzzfGOnF', 'type': 'tool_call'}],

In [9]:
graph.invoke({"messages": {"role": "user", "content": "마이크로소프트는 어떤 회사야?"}})

{'messages': [HumanMessage(content='마이크로소프트는 어떤 회사야?', additional_kwargs={}, response_metadata={}, id='e78ae2ba-0172-4be0-9d50-eb85fadae60c'),
  AIMessage(content='마이크로소프트(Microsoft)는 미국의 다국적 기술 회사로, 소프트웨어, 하드웨어, 전자기기 및 서비스 분야에서 활동하고 있습니다. 1975년에 빌 게이츠(Bill Gates)와 폴 앨런(Paul Allen)에 의해 설립되었으며, 현재는 세계에서 가장 큰 소프트웨어 회사 중 하나로 알려져 있습니다.\n\n주요 제품 및 서비스에는 다음이 포함됩니다:\n\n1. **운영 체제**: Windows 운영 체제는 개인용 컴퓨터와 서버에서 널리 사용됩니다.\n2. **오피스 소프트웨어**: Microsoft Office 제품군(Word, Excel, PowerPoint 등)은 비즈니스와 개인 사용자에게 널리 사용됩니다.\n3. **클라우드 서비스**: Microsoft Azure는 클라우드 컴퓨팅 서비스로, 기업들이 데이터 저장 및 처리, 애플리케이션 호스팅 등을 할 수 있게 지원합니다.\n4. **게임**: Xbox 게임 콘솔과 Xbox Live 서비스는 게임 산업에서도 중요한 역할을 하고 있습니다.\n5. **하드웨어**: Surface 시리즈와 같은 태블릿 및 PC를 포함한 다양한 하드웨어 제품을 제공합니다.\n\n마이크로소프트는 기술 혁신, 인공지능, 클라우드 컴퓨팅, 그리고 다양한 소프트웨어 솔루션을 통해 전 세계적으로 큰 영향력을 미치고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 299, 'prompt_tokens': 89, 'total_tokens': 388, 'completion_tokens_details': {'accepted_